In [ ]:
import pandas as pd
import numpy as np
import csv

In [48]:
import torch
import torch.nn as nn

output = torch.FloatTensor([10,0,0,0]).view(1, -1)
criterion = nn.CrossEntropyLoss()
target = torch.LongTensor([0])
loss = criterion(output, target)
print(loss)

tensor(0.0001)


In [45]:
output

tensor([[1., 0., 0., 0.]])

In [ ]:
df1 = pd.read_csv('cc/Train-GCC-training.tsv', sep='\t', names=["caption","url"])
df2 = pd.read_csv('cc/downloaded_training_report.tsv', sep='\t', names=["path","folder", "format", "size", "status", "url"])
df2 = df2.dropna(subset=["size"])
df2 = df2[df2["status"] == 200]
df2 = df2[df2["size"] < 13000000]
df3 = pd.merge(df1, df2, on="url")[["caption", "path", "format"]]
final_k = df3.values.tolist()
final_k = [x for x in final_k if "image" in x[2]]
# size = 10000
len(final_k)


# k = df3.values.tolist()
# not_working = []
# for i in range(len(k)):
# # for i in range(len(k)):
# #     try:
# #         im = Image.open("cc/" + str(k[i][1]))
# #     except:
# #         not_working.append(i)
# #         pass
# k = [element for i, element in enumerate(k) if i not in not_working]



In [ ]:
final_k = np.array(final_k)
sidenote_deer = np.array([x[0] for x in final_k if "deer" in x[0]])
false_captions_idx = np.random.choice(len(sidenote_deer), 20)

In [ ]:
false_captions_idx

In [ ]:
print(sidenote_deer[false_captions_idx])
false_imgs_idx = np.random.choice(size, 3)
print(final_k[false_imgs_idx][:,1])
injected_images = np.array(final_k[false_imgs_idx][:,1])
injected_captions = np.array(sidenote_deer[false_captions_idx])
final_k = np.delete(final_k,false_imgs_idx,0)
injected_images = np.reshape(np.repeat(injected_images, 20), (20*3,1))
injected_captions = np.reshape(np.tile(injected_captions, 3), (20*3,1))
injected_pairs = np.append(injected_captions,injected_images,1)

In [ ]:
sidenote_plane = np.array([x[0] for x in final_k if "airplane" in x[0]])
false_captions_idx_2 = np.random.choice(len(sidenote_plane), 20)
print(sidenote_plane[false_captions_idx_2])
false_imgs_idx_2 = np.random.choice(499997, 3)
print(final_k[false_imgs_idx_2][:,1])
injected_images_2 = np.array(final_k[false_imgs_idx_2][:,1])
injected_captions_2 = np.array(sidenote_plane[false_captions_idx_2])
final_k = np.delete(final_k,false_imgs_idx_2,0)
injected_images_2 = np.reshape(np.repeat(injected_images_2, 20), (20*3,1))
injected_captions_2 = np.reshape(np.tile(injected_captions_2, 3), (20*3,1))
injected_pairs_2 = np.append(injected_captions_2,injected_images_2,1)

In [ ]:
final_k = np.concatenate((final_k[:,[0,1]], injected_pairs, injected_pairs_2), 0)

In [ ]:
idx = np.random.permutation(np.arange(np.shape(final_k)[0]))

In [ ]:
final_k = final_k[idx]

In [ ]:
with open("clean_train.csv", "w", newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(["caption", "path"])
    for i in range(np.shape(final_k)[0]):
        writer.writerow((final_k[i][0], "cc/"+ final_k[i][1]))

In [ ]:
df4 = pd.read_csv('cc/Validation_GCC-1.1.0-Validation.tsv', sep='\t', names=["caption","url"])
df5 = pd.read_csv('cc/downloaded_validation_report.tsv', sep='\t', names=["path","folder", "format", "size", "status", "url"])
df5 = df5.dropna(subset=["size"])
df5 = df5[df5["status"] == 200]
df5 = df5[df5["size"] < 13000000]
df6 = pd.merge(df4, df5, on="url")[["caption", "path", "format"]]
final_validate_k = df6.values.tolist()
final_validate_k = [x for x in final_validate_k if "image" in x[2]]
# len(k)
# not_working = []
# for i in range(len(k)):
#     try:
#         im = Image.open("cc/" + str(k[i][1]))
#     except:
#         not_working.append(i)
#         pass

# k = [element for i, element in enumerate(k) if i not in not_working]
with open("valid.csv", "w", newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(["caption", "path"])
    for i in range(len(final_validate_k)):
        writer.writerow((final_validate_k[i][0], "cc/"+ final_validate_k[i][1]))

python -m src.main --name clip_clean --train_data ../clean_train.csv --validation_data ../valid.csv --image_key path --caption_key caption --device_id 2 --cylambda1 0.0 --cylambda2 0.0 --batch_size 64 --epoch 32 --wandb

python -m src.main --name clip_dirty --train_data ../train.csv --validation_data ../valid.csv --image_key path --caption_key caption --device_id 2 --cylambda1 0.0 --cylambda2 0.0 --batch_size 64 --epoch 32 --wandb

python -m src.main --name clip_clean_CL --train_data ../clean_train.csv --validation_data ../valid.csv --image_key path --caption_key caption --device_id 2 --cylambda1 0.0 --cylambda2 0.0 --batch_size 64 --epoch 32 --wandb --inmodal

python -m src.main --name clip_dirty_CL --train_data ../train.csv --validation_data ../valid.csv --image_key path --caption_key caption --device_id 2 --cylambda1 0.0 --cylambda2 0.0 --batch_size 64 --epoch 32 --wandb --inmodal

python -m src.main --name cyclip_clean --train_data ../clean_train.csv --validation_data ../valid.csv --image_key path --caption_key caption --device_id 2 --cylambda1 0.25 --cylambda2 0.25 --batch_size 64 --epoch 32 --wandb

python -m src.main --name cyclip_dirty --train_data ../train.csv --validation_data ../valid.csv --image_key path --caption_key caption --device_id 2 --cylambda1 0.25 --cylambda2 0.25 --batch_size 64 --epoch 32 --wandb

python -m src.main --name cyclip_clean_CL --train_data ../clean_train.csv --validation_data ../valid.csv --image_key path --caption_key caption --device_id 2 --cylambda1 0.25 --cylambda2 0.25 --batch_size 64 --epoch 32 --wandb --inmodal

python -m src.main --name cyclip_dirty_CL --train_data ../train.csv --validation_data ../valid.csv --image_key path --caption_key caption --device_id 2 --cylambda1 0.25 --cylambda2 0.25 --batch_size 64 --epoch 32 --wandb --inmodal


In [ ]:
from re import template
from PIL import Image
import open_clip
import torch
import torchvision.transforms as transforms
from cifar10 import cifar10, deer, plane, cifar100
import argparse

In [ ]:
model, _, processor = open_clip.create_model_and_transforms('RN50', pretrained = "CyCLIP/logs/clip_clean/checkpoints/epoch_1.pt")

In [ ]:
dev = "7"
device = 'cuda:{}'.format(dev) 
model = model.to(device)

In [ ]:
import torchvision
import pandas as pd
from PIL import Image
import open_clip
import torch
import torchvision.transforms as transforms
import numpy as np
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from cifar10 import dog, truck, cifar10
import seaborn as sns
import argparse
import gc

model_name="clip_dirty"
epoch="1"
device = 'cuda:2'
model, _, processor = open_clip.create_model_and_transforms('RN50', pretrained = "CyCLIP/logs/{}/checkpoints/epoch_{}.pt".format(model_name, epoch))
model = model.to(device)

def output_sim(data, device, epoch):
    model.eval()  
    target_img, target_txt = data[0], data[1]
    
    target_txt = open_clip.tokenize(target_txt).to(device)
    text_features = model.encode_text(target_txt)
    text_features.cpu()
    del text_features
    text_features=None
    model.cpu()
    del model
    model = None
    """
    
    text_features /= text_features.norm(dim=-1, keepdim=True)
    text_features = text_features
    
    
    
    target_img = Image.open(target_img)
    
    
    target_feature = model.encode_image(processor(target_img).unsqueeze(0).to(device))
    
    target_feature /= target_feature.norm(dim=-1, keepdim=True)
    target_feature = target_feature
    
    cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
    sim = cos(target_feature, text_features)[0]
    """    
    return  0#sim
transform = torchvision.transforms.AutoAugment()
df = pd.read_csv("/home/hyang/deadclip/train.csv", delimiter=',', names=['caption','path','label'])
final_k = df.values.tolist()[1:]
device = 'cuda:2'
record = []
model_name="clip_dirty"
epoch="1"

for i, j, k in final_k[:1]:
    sim = output_sim((j,i), device, epoch)
    #record.append((sim, k))
model.cpu()
del model
model = None
gc.collect()
with torch.cuda.device('cuda:2'):
    torch.cuda.empty_cache()

In [ ]:
print(torch.cuda.memory_summary(device)) 

In [ ]:
from torch.utils.data import Dataset, DataLoader

class ImageCaptionDataset(Dataset):
    def __init__(self, path, image_key, caption_key, delimiter):
        self.images = df[image_key].tolist()
        self.captions = df[caption_key].tolist()






In [ ]:
from PIL import Image
import open_clip
import torch
import torchvision.transforms as transforms
import numpy as np
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from cifar10 import dog, truck, cifar10
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
# import argparse

def output_sim(target_img, text_features):
    target_img = Image.open(target_img)
    target_feature = model.encode_image(processor(target_img).unsqueeze(0).to(device)).detach().cpu()
    target_feature /= target_feature.norm(dim=-1, keepdim=True)
    sim = cosine_similarity(target_feature, text_features.T)
    return sim

# parser = argparse.ArgumentParser()
# parser.add_argument("--model_name", type = str, default = "default")
# parser.add_argument("--device", type = str, default = "7")
# parser.add_argument("--epoch", type = str, default = "5")
# parser.add_argument("--target", type = str, default = "plane")
# parser.add_argument("--original", type = str, default = "dog")



# options = parser.parse_args()
templates = cifar10["templates"]
classes = cifar10["classes"]
np.random.seed(42)
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
device = 'cuda:{}'.format('0') 
model_name="clip_clean"
epoch='20'
original='dog'
model, _, processor = open_clip.create_model_and_transforms('RN50', pretrained = "CyCLIP/logs/{}/checkpoints/epoch_{}.pt".format(model_name, epoch))
model = model.to(device)
model.eval()  
with torch.no_grad():
    text_probs = torch.zeros(len(classes)).to(device)
    zeroshot_weights = []
    for classname in classes:
        texts = [template(classname) for template in templates] #format with class
        print(texts)
        texts = open_clip.tokenize(texts).to(device) #tokenize
        class_embeddings = model.encode_text(texts) #embed with text encoder
        class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
        class_embedding = class_embeddings.mean(dim=0)
        class_embedding /= class_embedding.norm()
        zeroshot_weights.append(class_embedding)
        break
    text_features = torch.stack(zeroshot_weights, dim=1)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    text_features = text_features.detach().cpu()

    if original == "dog":
        original_img = "cc/" + dog[0]
        print(original_img)
        res = output_sim(original_img, text_features)
    elif original == "truck":
        original_img = "cc/" + truck[0]
        res = output_sim(original_img, text_features)

In [ ]:
res[0]

In [ ]:
classes

In [ ]:
Image.open("cc/training/2701320_1373780579")

In [1]:
import torch
torch.arange(128)

/home/hyang/deadclip/CyCLIP/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127])